In [1]:
import cv2
import os

# Initialize webcam
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Create dataset folder
dataset_path = "dataset"
if not os.path.exists(dataset_path):
    os.makedirs(dataset_path)

# Ask for user name
user_name = input("Enter your name: ")
user_folder = os.path.join(dataset_path, user_name)

if not os.path.exists(user_folder):
    os.makedirs(user_folder)

print("Capturing images... Press 'q' to quit.")

count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        count += 1
        face_img = gray[y:y+h, x:x+w]
        cv2.imwrite(f"{user_folder}/{count}.jpg", face_img)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow("Capturing Faces", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q') or count >= 100:
        break

cap.release()
cv2.destroyAllWindows()
print(f"Captured {count} images for {user_name}.")

2025-03-29 18:21:27.989 Python[10407:163682] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


Capturing images... Press 'q' to quit.


2025-03-29 18:21:34.716 Python[10407:163682] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-29 18:21:34.716 Python[10407:163682] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Captured 100 images for Neev Gupta.


In [2]:
import cv2
import os
import numpy as np
import pickle

# Ensure OpenCV contrib module is installed
if not hasattr(cv2.face, "LBPHFaceRecognizer_create"):
    raise ImportError("OpenCV face module is missing! Install with: pip install opencv-contrib-python")

# Initialize the Face Recognizer
recognizer = cv2.face.LBPHFaceRecognizer_create()

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Path to dataset folder
dataset_path = "dataset"
faces = []
labels = []
label_dict = {}
current_id = 0

# Process each person's folder in the dataset
for person_name in os.listdir(dataset_path):
    person_path = os.path.join(dataset_path, person_name)
    if not os.path.isdir(person_path):
        continue

    # Assign an ID to each person
    if person_name not in label_dict:
        label_dict[person_name] = current_id
        current_id += 1

    person_id = label_dict[person_name]

    # Process each image in the person's folder
    for image_name in os.listdir(person_path):
        image_path = os.path.join(person_path, image_name)
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        if img is None:
            print(f"Skipping invalid image: {image_path}")
            continue

        faces.append(img)
        labels.append(person_id)

# Convert lists to numpy arrays
faces = np.array(faces, dtype="object")
labels = np.array(labels)

# Ensure training data is not empty
if len(faces) == 0:
    raise ValueError("Error: No valid face images found for training!")

print(f"Training on {len(faces)} images from {len(label_dict)} people...")

# Train the recognizer
recognizer.train(faces, labels)

# Save the trained model
recognizer.save("face_model.yml")

# Save label dictionary
with open("labels.pkl", "wb") as f:
    pickle.dump(label_dict, f)

print("Training complete! Model and labels saved.")

Training on 301 images from 3 people...
Training complete! Model and labels saved.


In [ ]:
import cv2
import pickle
import numpy as np
from deepface import DeepFace

# Convert the image to RGB (DeepFace requires RGB)
face_rgb = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)

# Analyze emotion using DeepFace
analysis = DeepFace.analyze(face_rgb, actions=['emotion'], enforce_detection=False)

# Extract the dominant emotion
detected_emotion = analysis[0]['dominant_emotion'] if analysis else "Unknown"

print(f"Detected Emotion: {detected_emotion}")  # Debugging output
# Load trained face recognition model
recognizer = cv2.face.LBPHFaceRecognizer_create()
try:
    recognizer.read("face_model.yml")
    print("Face model loaded successfully!")
except Exception as e:
    print("Error loading face model:", str(e))
    exit()

# Load label dictionary
try:
    with open("labels.pkl", "rb") as f:
        label_dict = pickle.load(f)
    print("Label dictionary loaded:", label_dict)  # Debugging output
except Exception as e:
    print("Error loading labels:", str(e))
    exit()

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not capture frame!")
        break

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(50, 50))

    for (x, y, w, h) in faces:
        face_img = gray[y:y+h, x:x+w]  # Extract face region

        # Recognize the face
        label_id, confidence = recognizer.predict(face_img)
        print(f"Predicted ID: {label_id}, Confidence: {confidence}")  # Debugging output

        # Dynamically adjust threshold based on testing
        ADAPTIVE_THRESHOLD = 60  # You can tune this between 50-70

        # Determine the name
        name = "Unknown"
        if label_id in label_dict.values() and confidence < ADAPTIVE_THRESHOLD:
            # Get name from the label dictionary
            name = [k for k, v in label_dict.items() if v == label_id][0]
        else:
            name = "Unknown"  # If confidence is too high, treat as unknown
        # Detect emotions
        try:
            emotion_analysis = DeepFace.analyze(frame[y:y+h, x:x+w], actions=['emotion'], enforce_detection=False)
            emotion = emotion_analysis[0]['dominant_emotion']
            print(f"Detected Emotion: {emotion}")
        except Exception as e:
            print("Emotion Detection Error:", str(e))
            emotion = "Unknown"

        # Draw rectangle & text
        # Define text
        window_height, window_width, _ = frame.shape
        text = "Person: "+name+" | Emotion: "+detected_emotion
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 2
        font_thickness = 4
        text_color = (0, 102, 204) 

        # Get text size for positioning
        (text_width, text_height), _ = cv2.getTextSize(text, font, font_scale, font_thickness)

        # Position text at the **bottom center** of the window
        text_x = (window_width - text_width) // 2  # Center horizontally
        text_y = window_height - 50  # 20 pixels from the bottom

        # Draw text (separate from the face detection logic)
        cv2.putText(frame, text, (text_x, text_y), font, font_scale, text_color, font_thickness)

        color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        # cv2.putText(frame, f"{name} ({emotion})", (x, y - 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.putText(frame, f"{name}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        

    # Display the frame
    cv2.imshow("Face & Emotion Recognition", frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()

Detected Emotion: neutral
Face model loaded successfully!
Label dictionary loaded: {'Sannidhya': 0, 'Neev Gupta': 1, 'Aakarsh': 2}
Predicted ID: 1, Confidence: 11.141453596629384
Detected Emotion: neutral
Predicted ID: 1, Confidence: 8.90101142042911
Detected Emotion: fear
Predicted ID: 1, Confidence: 9.108075596005055
Detected Emotion: neutral
Predicted ID: 1, Confidence: 8.986917785858758
Detected Emotion: neutral
Predicted ID: 1, Confidence: 7.977015913873518
Detected Emotion: neutral
Predicted ID: 1, Confidence: 7.962836288495423
Detected Emotion: neutral
Predicted ID: 1, Confidence: 8.802961490694234
Detected Emotion: neutral
Predicted ID: 1, Confidence: 8.071278870415085
Detected Emotion: neutral
Predicted ID: 1, Confidence: 8.637377629097386
Detected Emotion: neutral
Predicted ID: 1, Confidence: 8.487241242515847
Detected Emotion: neutral
Predicted ID: 1, Confidence: 8.182341250500711
Detected Emotion: neutral
Predicted ID: 1, Confidence: 8.126466390489053
Detected Emotion: neut